In [ ]:
import torch
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets
import os
import pandas as pd
from torchvision.io import read_image
import tarfile
from PIL import Image
import pickle
import cv2
import scipy
from tqdm import tqdm
from scipy import spatial

In [ ]:
def get_file_list(file_path = "./image_data/"):
    import random
    file_list = []
    assert os.path.isdir(file_path)
    for file in os.listdir(file_path):
        file_list.append(file_path+file)
    return file_list

In [ ]:
class Collection(Dataset):
    def __init__(self, image_path, transform=None):
        super().__init__()
        self.image_path = image_path
        self.transform = transform
    def __getitem__(self, index):
        image_path = self.image_path[index]
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, image_path
    def __len__(self):
        return len(self.image_path)

In [ ]:
def load_data(data_path, batch_size = 4, shuffle=False, transform='default'):
    default_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.225, 0.225, 0.225))
    ]) if transform == 'default' else transform

    image_path_list = get_file_list(file_path=data_path)
    
    collection = Collection(image_path=image_path_list, transform = default_transform)

    dataloader = DataLoader(dataset=collection, batch_size=batch_size, shuffle=shuffle, num_workers=0)
    return dataloader

In [ ]:
def feature_extract(model, dataloader):
    features = torch.FloatTensor()
    path_list = []
    for img, path in dataloader:
        img = img.cuda()
        input = Variable(img)
        output = model(input)
        feature = output.data.cpu()
        feature_norm = torch.norm(feature, p=2, dim=1, keepdim=True)
        feature = feature.div(feature_norm.expand_as(feature))
        features = torch.cat((features, feature), 0)
        path_list += list(path)
    return features, path_list

In [ ]:
def create_query_folder(path = "/content/drive/MyDrive/cs336/image_annotation"):
    query_list = []
    file_name = []
    for query in os.listdir(path):
        if query.split('.')[0].split('_')[-1] == "query":
            file_name.append(query.split('.')[0] + '.jpg')
            file = open(path+query, 'r')
            content = file.readline().split()
            crop_img, x, y, w, h = content
            file.close()
            crop_img = crop_img.split('_')[1::]
            crop_img = '_'.join(crop_img)
            query_list.append((crop_img + '.jpg', x, y, w, h))
    query_dir = "./image_data/"
    query_des = "./image_query/"
    for item in range(len(query_list)):
        img = cv2.imread(query_dir+query_list[item][0])
        x, y, w, h = float(query_list[item][1]), float(query_list[item][2]), float(query_list[item][3]), float(query_list[item][4])
        img = img[int(x):int(x+w), int(y):int(y+h)]
        cv2.imwrite(query_des+file_name[item], img)

In [ ]:
def load_model_vgg():
    model = models.vgg16(pretrained=True)
    model.classifier = model.classifier[:-1]
    model = model.eval()
    model = model.cuda()
    return model

In [ ]:
def load_model_resnet():
    model = models.resnet101(pretrained=True)
    modules = list(model.children())[:-1]
    model = nn.Sequential(*modules)
    model = model.cuda()
    model = model.eval()
    return model

In [ ]:
def compute_score(query_feature, collection_feature):
    score = [spatial.distance.cosine(query_feature, feature) for feature in collection_feature]
    idx = np.argsort(score, axis=0)
    score = sorted(score)
    return score, idx

In [ ]:
model2 = load_model_resnet()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [ ]:
model = load_model_vgg()

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
#ataloader = load_data("/content/drive/MyDrive/cs336/data/", batch_size=8, shuffle=False, transform='default')

In [ ]:
#collection, image_paths = feature_extract(model=model, dataloader=dataloader)

In [ ]:
#q_dataloader = load_data("/content/drive/MyDrive/cs336/image_query/", batch_size=1, shuffle=False, transform='default')
#query_feature, q_path = feature_extract(model=model, dataloader=q_dataloader)

In [ ]:
#os.listdir('/content/drive/MyDrive/cs336/image_annotation/')

In [ ]:
#q_path

In [ ]:
'''dir = ["./collection_features.pkl", "./collection_paths.txt", "./query_features.pkl", "./query_paths.txt"]
content = [collection, image_paths, query_feature, q_path]
for i in range(len(dir)):
    file = open(dir[i], "wb")
    pickle.dump(content[i], file)
    file.close()'''

'dir = ["./collection_features.pkl", "./collection_paths.txt", "./query_features.pkl", "./query_paths.txt"]\ncontent = [collection, image_paths, query_feature, q_path]\nfor i in range(len(dir)):\n    file = open(dir[i], "wb")\n    pickle.dump(content[i], file)\n    file.close()'

In [ ]:

#collection = pickle.load(open("/content/drive/MyDrive/cs336/features/data/collection_features.pkl", "rb"))
#image_paths = pickle.load(open("/content/drive/MyDrive/cs336/features/data/collection_paths.txt", "rb"))
#query_feature = pickle.load(open("/content/query_features.pkl", "rb"))
#q_path = pickle.load(open("/content/query_paths.txt", "rb")

In [ ]:
def load_query(img_path, batch_size = 1, shuffle=False, transform='default'):
    default_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.225, 0.225, 0.225))
    ]) if transform == 'default' else transform
    
    collection = Collection(image_path=[img_path], transform = default_transform)

    dataloader = DataLoader(dataset=collection, batch_size=batch_size, shuffle=shuffle, num_workers=0)
    return dataloader


In [ ]:
def retrieval(img_path):
    collection_feature = None
    collection_path = None
    
    #m = load_model_vgg()
    m= model
    dataloader =load_query(img_path)
    query_feature, query_path = feature_extract(m, dataloader)
 
    with open('/content/drive/MyDrive/cs336/features/data/collection_features.pkl', 'rb') as pickle_file:
        collection_feature = pickle.load(pickle_file)
    
    with open('/content/drive/MyDrive/cs336/features/data/collection_paths.txt', 'rb') as pickle_file:
        collection_path = pickle.load(pickle_file)
    # collection_feature, collection_path = pickle.load('vgg16_extractor_result/collection_features.pkl'), pickle.load('vgg16_extractor_result/collection_paths.txt')
    similarity, index = compute_score(query_feature, collection_feature)
    sorted_paths = [collection_path[i] for i in index]
    return sorted_paths, similarity


In [ ]:
def retrieval2(img_path):
    collection_feature = None
    collection_path = None
    
    #m = load_model_vgg()
    m= model2
    dataloader =load_query(img_path)
    query_feature, query_path = feature_extract(m, dataloader)
 
    with open('/content/drive/MyDrive/cs336/features/data/collection_features2.pkl', 'rb') as pickle_file:
        collection_feature = pickle.load(pickle_file)
    
    with open('/content/drive/MyDrive/cs336/features/data/collection_paths2.txt', 'rb') as pickle_file:
        collection_path = pickle.load(pickle_file)
    # collection_feature, collection_path = pickle.load('vgg16_extractor_result/collection_features.pkl'), pickle.load('vgg16_extractor_result/collection_paths.txt')
    similarity, index = compute_score(query_feature, collection_feature)
    sorted_paths = [collection_path[i] for i in index]
    return sorted_paths, similarity

In [ ]:
#a,s = retrieval('/content/drive/MyDrive/cs336/testimg/Copy of hertford_4_query.jpg')

In [ ]:
#a[0:10]

In [ ]:
'''ig=cv2.imread('/content/drive/MyDrive/cs336/testimg/Copy of hertford_4_query.jpg')
ig1 = cv2.imread(a[0])
ig2 = cv2.imread(a[1])
cv2_imshow(ig)
cv2_imshow(ig1)
cv2_imshow(ig2)'''

"ig=cv2.imread('/content/drive/MyDrive/cs336/testimg/Copy of hertford_4_query.jpg')\nig1 = cv2.imread(a[0])\nig2 = cv2.imread(a[1])\ncv2_imshow(ig)\ncv2_imshow(ig1)\ncv2_imshow(ig2)"

In [ ]:
#a,s = retrieval('/content/drive/MyDrive/cs336/data/all_souls_000003.jpg')

In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 41.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=73bd249fd1daab1526d65a82a44884c0934b3d1d770c708bd0eb1edf8b9e1977
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [ ]:
import getpass

from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = '2H7faLYvDQGs7VkU4ejNQu7i26f_76et6FX9v9PF5VBAqGJ7z'

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect(22, "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
 * ngrok tunnel available, access with `ssh root@2.tcp.ngrok.io -p14257`


In [ ]:
!pip install -U flask-cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import threading
from flask import Flask, jsonify
from flask_cors import CORS,cross_origin

from flask import request
from pyngrok import ngrok
app = Flask(__name__)

cors = CORS(app, resources={r"/api/*": {"origins": "*"}})
port = 5000
os.environ["FLASK_ENV"] = "development"
# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url
UPLOAD_FOLDER = 'static/uploads/'
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024
 
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg', 'gif'])
 
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/todo/api/v1.0/tasks', methods=['GET'])
@cross_origin() 
def get_tasks():
    a,s = retrieval('/content/drive/MyDrive/cs336/data/all_souls_000003.jpg')
    return jsonify({'tasks': a[0:10]})

@app.route('/todo/api/v1.0/tasks', methods=['POST'])
@cross_origin()
def create_task():
    if(request.json['m'] == 'vgg'):
      a,s = retrieval('/content/drive/MyDrive/cs336/data/' + request.json['url'])
      return jsonify({'tasks': a[0:10]})
    else : 
      a,s = retrieval2('/content/drive/MyDrive/cs336/data/' + request.json['url'])
      return jsonify({'tasks': a[0:10]})
   # url =  request.json,
    #a =  request.json['url']
    #return jsonify(url), 201
# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

 * ngrok tunnel "http://cf56-35-247-147-106.ngrok.io" -> "http://127.0.0.1:5000"
 * Serving Flask app "__main__" (lazy loading)
 * Environment: development
 * Debug mode: on


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
